In [1]:
import yfinance as yf
import os, contextlib
import pandas as pd
from pathlib import Path
import tqdm

**Get SP500 list**

In [5]:
def get_sp500_tickers_alternative():

    # fetch SP500
    csv_url = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv'
    df = pd.read_csv(csv_url)

    tickers = df['Symbol'].tolist()
    tickers = [ticker.replace('.', '-') for ticker in tickers]
    
    return tickers


sp500_symbols = get_sp500_tickers_alternative()

# 打印结果进行验证
print("length of sp500_symbols:", len(sp500_symbols))
print(sp500_symbols[:10])

length of sp500_symbols: 503
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


**Use yfinance API to get the stock data in SP500 list from 2021 to now**

In [95]:
outfile = Path('sp500_yfinance_data.csv')
if outfile.exists():
    outfile.unlink()

In [ ]:
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in tqdm.tqdm(range(len(sp500_symbols))):
            s = sp500_symbols[i]
            # avoid multi-level index
            data = yf.download(s, start='2021-01-01', end='2025-11-07', multi_level_index = False)
            # WBA will be skipped due to no data
            if len(data.index) == 0:
                continue
            df = data.reset_index()
            # insert code column
            df.insert(0, "code", s)
            
            # append to csv, only write header if file does not exist
            df.to_csv(outfile, index=False, mode="a", header=not outfile.exists())
            
print(f"All data saved to: {outfile.resolve()}")

  0%|          | 0/503 [00:00<?, ?it/s]/var/folders/4k/11kxhlxx3bl54nl7931sq1w80000gn/T/ipykernel_5469/2256365825.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(s, start='2021-01-01', end='2025-11-07', multi_level_index = False)
[*********************100%***********************]  1 of 1 completed
  0%|          | 1/503 [00:00<01:57,  4.28it/s]/var/folders/4k/11kxhlxx3bl54nl7931sq1w80000gn/T/ipykernel_5469/2256365825.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(s, start='2021-01-01', end='2025-11-07', multi_level_index = False)
[*********************100%***********************]  1 of 1 completed
  0%|          | 2/503 [00:00<01:36,  5.20it/s]/var/folders/4k/11kxhlxx3bl54nl7931sq1w80000gn/T/ipykernel_5469/2256365825.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(s, start='2021-01-01', end='2025-11-07', multi_level_

All data saved to: /Users/zhangyici/Desktop/研一 Fall/DB/Final_Project/StockScope/etl/sp500_yfinance_data.csv


In [6]:
dat = yf.Ticker("AMZN")
dat.info

{'address1': '410 Terry Avenue North',
 'city': 'Seattle',
 'state': 'WA',
 'zip': '98109-5210',
 'country': 'United States',
 'phone': '206 266 1000',
 'website': 'https://www.amazon.com',
 'industry': 'Internet Retail',
 'industryKey': 'internet-retail',
 'industryDisp': 'Internet Retail',
 'sector': 'Consumer Cyclical',
 'sectorKey': 'consumer-cyclical',
 'sectorDisp': 'Consumer Cyclical',
 'longBusinessSummary': "Amazon.com, Inc. engages in the retail sale of consumer products, advertising, and subscriptions service through online and physical stores in North America and internationally. The company operates through three segments: North America, International, and Amazon Web Services (AWS). It also manufactures and sells electronic devices, including Kindle, fire tablets, fire TVs, echo, ring, blink, and eero; and develops and produces media content. In addition, the company offers programs that enable sellers to sell their products in its stores; and programs that allow authors, 

**Get Company Information CSV**

In [87]:
import csv

In [88]:
outfile = Path('sp500_company_data.csv')
header = [
    "code", "name", "full_time_employees",
    "long_business_summary", "industry", "city", "state", "country"
]
if not outfile.exists():
    with open(outfile, "w", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(header)

In [89]:
# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in tqdm.tqdm(range(len(sp500_symbols))):
            s = sp500_symbols[i]
            data = yf.Ticker(s).info
            row = [
                s, # code
                data.get("longName", "hahaha"), # name, make sure name exists
                data.get("fullTimeEmployees"),
                data.get("longBusinessSummary"),
                data.get("industry"),
                data.get("city"),
                data.get("state"),
                data.get("country")
            ]
            with open(outfile, "a", newline='', encoding="utf-8") as f:
                writer = csv.writer(f)
                writer.writerow(row)
            
print(f"All data saved to: {outfile.resolve()}")

100%|██████████| 503/503 [01:40<00:00,  5.03it/s]

All data saved to: /Users/zhangyici/Desktop/研一 Fall/DB/Final_Project/StockScope/etl/sp500_company_data.csv


**Sentiment Score**

In [2]:
import requests
import csv
import time
from datetime import datetime
from pathlib import Path
import yfinance as yf
import pandas as pd

/Users/jeffvan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# get sp500 list
def get_sp500_tickers_alternative():

    # fetch SP500
    csv_url = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv'
    df = pd.read_csv(csv_url)

    tickers = df['Symbol'].tolist()
    tickers = [ticker.replace('.', '-') for ticker in tickers]
    
    return tickers


sp500_symbols = get_sp500_tickers_alternative()

# validation
print("length of sp500_symbols:", len(sp500_symbols))
print(sp500_symbols[:10])

length of sp500_symbols: 503
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


In [4]:
# basic info
# API_KEY = "4WH041V75ZSYLZDY" #ZYC's API key

# API_KEY = "5IOK22YFLKBUDHL"         # Jack's API key
API_KEY = "LONB725JY0NFBGY6"          # WXY's
# API_KEY = "8TOEQYXQGWV486I1"    #XZJ
TIME_FROM = "20210101T0130"
LIMIT = 1000
BASE_URL = "https://www.alphavantage.co/query"

In [5]:
sp500_set = set(sp500_symbols)
NEWS_DETAIL_CSV = Path("news_detail.csv")
NEWS_COMPANY_SENTIMENT_CSV = Path("news_company_sentiment.csv")
REQUEST_SLEEP_SECONDS = 15

In [6]:
TODAY_TICKERS = sp500_symbols[225:250]


In [7]:
TODAY_TICKERS

['HCA',
 'DOC',
 'HSIC',
 'HSY',
 'HPE',
 'HLT',
 'HOLX',
 'HD',
 'HON',
 'HRL',
 'HST',
 'HWM',
 'HPQ',
 'HUBB',
 'HUM',
 'HBAN',
 'HII',
 'IBM',
 'IEX',
 'IDXX',
 'ITW',
 'INCY',
 'IR',
 'PODD',
 'INTC']

In [8]:
def parse_timestamp(time_str: str) -> str:
    """
    transform to SQL timestamp
    input format: '20251116T140100'
    return: '2025-11-16 14:01:00'
    """
    try:
        dt = datetime.strptime(time_str, "%Y%m%dT%H%M%S")
        return dt.strftime("%Y-%m-%d %H:%M:%S")
    except Exception:
        return ""

In [9]:
def fetch_news_for_ticker(ticker: str):
    """
    get the json of news of one ticker
    """
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": ticker,
        "time_from": TIME_FROM,
        "limit": LIMIT,
        "apikey": API_KEY,
    }

    try:
        resp = requests.get(BASE_URL, params=params, timeout=30)
    except Exception as e:
        print(f"[ERROR] Request failed for {ticker}: {e}")
        return None

    if resp.status_code != 200:
        print(f"[ERROR] HTTP {resp.status_code} for {ticker}: {resp.text[:200]}")
        return None

    try:
        data = resp.json()
    except Exception as e:
        print(f"[ERROR] JSON decode failed for {ticker}: {e}")
        return None

    if "Error Message" in data or "Note" in data:
        print(f"[WARN] API error / note for {ticker}: {data.get('Error Message') or data.get('Note')}")
        return None

    return data

In [10]:
def load_existing_news():
    """
    get the current news_key_to_id: (url, time_published) -> id mapping from current csv file
    """
    news_key_to_id = {}
    max_id = 0

    if NEWS_DETAIL_CSV.exists():
        with NEWS_DETAIL_CSV.open("r", newline="", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                try:
                    nid = int(row["id"])
                except (KeyError, ValueError):
                    continue
                url = row.get("url")
                ts = row.get("timestamp_raw")
                if not url or not ts:
                    continue
                news_key_to_id[(url, ts)] = nid
                if nid > max_id:
                    max_id = nid

    next_news_id = max_id + 1 if max_id > 0 else 1
    return news_key_to_id, next_news_id

In [11]:
def load_existing_sentiments():
    """
    get current sentiment_written: set((company_code, news_id)) from current csv file
    """
    sentiment_written = set()
    if NEWS_COMPANY_SENTIMENT_CSV.exists():
        with NEWS_COMPANY_SENTIMENT_CSV.open("r", newline="", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                company_code = row.get("company_code")
                try:
                    news_id = int(row.get("news_id", ""))
                except (TypeError, ValueError):
                    continue
                if company_code and news_id:
                    sentiment_written.add((company_code, news_id))
    return sentiment_written

In [12]:
# load current information for avoiding duplication, calls every time as long as the csv file updates
news_key_to_id, next_news_id = load_existing_news()
sentiment_written = load_existing_sentiments()
print(f"Loaded {len(news_key_to_id)} existing news, next_news_id = {next_news_id}")
print(f"Loaded {len(sentiment_written)} existing sentiments, ready to go")

# append model, write header if the file does not exist
detail_file_exists = NEWS_DETAIL_CSV.exists()
sent_file_exists = NEWS_COMPANY_SENTIMENT_CSV.exists()

with NEWS_DETAIL_CSV.open("a", newline="", encoding="utf-8") as f_detail, \
     NEWS_COMPANY_SENTIMENT_CSV.open("a", newline="", encoding="utf-8") as f_sent:

    # to avoid duplication, add one more row timestamp_raw
    detail_fieldnames = ["id", "url", "title", "timestamp", "timestamp_raw"]
    sentiment_fieldnames = ["company_code", "news_id", "sentiment_score", "sentiment_level"]

    detail_writer = csv.DictWriter(f_detail, fieldnames=detail_fieldnames)
    sentiment_writer = csv.DictWriter(f_sent, fieldnames=sentiment_fieldnames)

    # the file does not exist, add header
    if not detail_file_exists:
        detail_writer.writeheader()
    if not sent_file_exists:
        sentiment_writer.writeheader()

    # get target tickers
    for idx, ticker in enumerate(TODAY_TICKERS, start=1):
        print(f"=== [{idx}/{len(TODAY_TICKERS)}] Fetching news for {ticker} ===")

        data = fetch_news_for_ticker(ticker)
        if not data:
            print(f"[WARN] Skip {ticker} due to error/empty response")
            time.sleep(REQUEST_SLEEP_SECONDS)
            continue

        feed = data.get("feed", [])
        print(f"Got {len(feed)} news items for {ticker}")

        for article in feed:
            url = article.get("url")
            title = article.get("title")
            time_published = article.get("time_published")

            if not url or not title or not time_published:
                continue

            news_key = (url, time_published)

            # !!!IMPORTANT!!! if it is first time to encounter this news, get a new news_id for this news
            if news_key not in news_key_to_id:
                news_id = next_news_id
                # store this news to news_id mapping
                news_key_to_id[news_key] = news_id
                next_news_id += 1

                ts_str = parse_timestamp(time_published)

                detail_writer.writerow({
                    "id": news_id,
                    "url": url,
                    "title": title,
                    "timestamp": ts_str,
                    "timestamp_raw": time_published,
                })
            else:
                news_id = news_key_to_id[news_key]

            # for each tickers sentiment score, write into NewsCompanySentiment
            # avoid duplication using sentiment_written
            ticker_sentiments = article.get("ticker_sentiment", [])
            for tsent in ticker_sentiments:
                company_code = tsent.get("ticker")
                if not company_code:
                    continue

                # only store company in sp500_set
                if company_code not in sp500_set:
                    continue

                score_str = tsent.get("ticker_sentiment_score")
                label = tsent.get("ticker_sentiment_label")

                if score_str is None or label is None:
                    continue

                try:
                    score = float(score_str)
                except ValueError:
                    continue
                
                # !!!IMPORTANT!!! avoid duplication on sentiment, avoid same company_code, news_id rows/pairs
                key = (company_code, news_id)
                # if duplicates, continue to next company 
                if key in sentiment_written:
                    continue
                
                sentiment_writer.writerow({
                    "company_code": company_code,
                    "news_id": news_id,
                    "sentiment_score": f"{score:.6f}",
                    "sentiment_level": label,
                })
                # add the new company_code, news_id pair
                sentiment_written.add(key)

        print(f"Sleeping {REQUEST_SLEEP_SECONDS} seconds to respect API rate limit...")
        time.sleep(REQUEST_SLEEP_SECONDS)

print("Done appending! Now the CSV fild has been updated: ")
print(f" - {NEWS_DETAIL_CSV.resolve()}")
print(f" - {NEWS_COMPANY_SENTIMENT_CSV.resolve()}")

Loaded 112291 existing news, next_news_id = 112292
Loaded 257176 existing sentiments, ready to go
=== [1/25] Fetching news for HCA ===
Got 748 news items for HCA
Sleeping 15 seconds to respect API rate limit...
=== [2/25] Fetching news for DOC ===
Got 87 news items for DOC
Sleeping 15 seconds to respect API rate limit...
=== [3/25] Fetching news for HSIC ===
Got 713 news items for HSIC
Sleeping 15 seconds to respect API rate limit...
=== [4/25] Fetching news for HSY ===
Got 715 news items for HSY
Sleeping 15 seconds to respect API rate limit...
=== [5/25] Fetching news for HPE ===
Got 648 news items for HPE
Sleeping 15 seconds to respect API rate limit...
=== [6/25] Fetching news for HLT ===
Got 696 news items for HLT
Sleeping 15 seconds to respect API rate limit...
=== [7/25] Fetching news for HOLX ===
Got 746 news items for HOLX
Sleeping 15 seconds to respect API rate limit...
=== [8/25] Fetching news for HD ===
Got 706 news items for HD
Sleeping 15 seconds to respect API rate limit.